In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import html
import re
import string
from nltk.corpus import stopwords
from gensim.summarization import summarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from gensim.summarization import summarize
from csv import DictReader
import nltk
nltk.download('stopwords')

# Keras Import Statements
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input, Lambda
from keras.layers.wrappers import Bidirectional
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD, Adagrad
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU
from keras.layers import concatenate,dot
from keras.preprocessing import sequence
import keras.backend as K
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import matplotlib.pyplot as plt
import pickle


# Access google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Loading Datasets

In [ ]:
class DataSet():
    def __init__(self, name="train", path="fnc-1"):
        self.path = path

        print("Reading dataset")
        bodies = name+"_bodies.csv"
        stances = name+"_stances.csv"

        self.stances = self.read(stances)
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))



    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows


In [ ]:
train_dataset = DataSet(name="train",path = "drive/My Drive/Stance_detection/data/")
test_dataset = DataSet(name="test",path = "drive/My Drive/Stance_detection/data/")

# Global Variables

In [ ]:
global_map = dict()
SUMMARY_LEN = 100

#Variables for Preprocessing
do_summary = True
head_stop,head_summary = True, False
body_stop,body_summary = True, False

# Embedding Dimension
EMBEDDING_DIM = 300

# Head and body max
max_head = 30
max_body = 100

# Function to Preprocess the data

In [ ]:
# Mean Layer
def adder(x):
    x = K.mean(x, axis=1)
    # x = K.reshape(x,(K.shape(x)[0],K.shape(x)[-1]))
    return x

def adder_output(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3  # only valid for 3D tensors
    shape = (shape[0],shape[2])
    return tuple(shape)

def preprocess(text,stop,do_summ):
    g_text = text
    if g_text in global_map :
        return global_map[g_text]
    
    if do_summ:
        temp = re.sub(r'[.]+',"\n",text)
        if len(temp.split()) > SUMMARY_LEN:
            text = summarize(temp,word_count = SUMMARY_LEN)
              
    text = html.unescape(text)
    text = text.replace("\\n"," ")
    text = text.replace("_NEG","")
    text = text.replace("_NEGFIRST", "")
    text = re.sub(r"#", "", text)
    text = re.sub(r"\*", "", text)
    text = re.sub(r"\'s", "", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r",", "", text)
    text = re.sub(r"!", " !", text)
    text = re.sub(r"\(", "", text)
    text = re.sub(r"\)", "", text)
    text = re.sub(r"\?", " ?", text)
    text = re.sub(r'[^\x00-\x7F]',' ', text)
    text = re.sub(r'[^\w\s]',' ',text)
    text = re.sub("\d+", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.rstrip(',|.|;|:|\'|\"')
    text = text.lstrip('\'|\"')
    if stop:
        temp = remove_stopwords(text.strip().lower())
        global_map[g_text] = temp
    else:
        temp = (text.strip().lower())
        global_map[g_text] = temp
    return global_map[g_text]

def remove_stopwords(text):
    temp = stopwords.words('english')
    split_text = \
    [word for word in text.split()
        if word not in temp]
    return " ".join(split_text)

def create_total_text(dataset,isStance,total_text):
    if isStance:
        for stance in dataset.stances:
            total_text.append(preprocess(stance['Headline'],head_stop,head_summary))
    else:
        for article_id in dataset.articles:
            total_text.append(preprocess(dataset.articles[article_id],body_stop,body_summary))

In [ ]:
# Total Dataset 
total_text = list()
create_total_text(train_dataset,True,total_text)
print(len(total_text)) #sanity check
create_total_text(train_dataset,False,total_text)
print(len(total_text)) #sanity check
create_total_text(test_dataset,True,total_text)
print(len(total_text)) #sanity check
create_total_text(test_dataset,False,total_text)
print(len(total_text)) #sanity check

# Fiting a tokenizer

In [ ]:
t = Tokenizer()
t.fit_on_texts(total_text)
word_index = t.word_index
print(t.document_count)
vocab_size = len(t.word_counts)
print(vocab_size)

# Loading work embedding

In [ ]:
temp_name = 'embedding_matrix_w2v'
create_w2vec = True
if do_summary:
    temp_name+="_summary.npy"
else:
    temp_name+="_no_summary.npy"
if create_w2vec:
    
    GLOVE_DIR = "drive/My Drive/Stance_detection/GoogleNews-vectors-negative300.txt"
    embeddings_index = {}
    f = open(GLOVE_DIR)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print ('Read Word2Vec and Made Dict')

    embedding_matrix = np.zeros((vocab_size + 1, EMBEDDING_DIM))
    number_found =0
    number_not_found = 0
    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
            number_found+=1
        else:
            #print (word)
            number_not_found+=1

    print(number_found)
    print(number_not_found)
    np.save(temp_name,embedding_matrix)
else:
    embedding_matrix = np.load(temp_name)

# Creating final dataset to be used in dnn models

In [ ]:
def create_dataset(data,ids,isTest,t,max_head,max_body):
    
    from keras.utils.np_utils import to_categorical
    head = list()
    body = list()
    y = list()
    NUM_CLASSES = 4
    category_dict = {'unrelated': 0 , 'agree':1, 'disagree':2, 'discuss':3}
    for stance in data.stances:
        if(isTest):
            head.append(preprocess(stance['Headline'],head_stop,head_summary))
            body.append(preprocess(data.articles[int(stance['Body ID'])],body_stop,body_summary))
            y.append(category_dict[stance['Stance']])
            continue
        
        if stance['Body ID'] not in ids:
            continue
        
        head.append(preprocess(stance['Headline'],head_stop,head_summary))
        body.append(preprocess(data.articles[int(stance['Body ID'])],body_stop,body_summary))
        y.append(category_dict[stance['Stance']])
    
    head = t.texts_to_sequences(head)
    body = t.texts_to_sequences(body)
    head = pad_sequences(head,maxlen = max_head,padding = 'post')
    body = pad_sequences(body,maxlen = max_body,padding = 'post')
    y_cat = np.zeros((len(y),NUM_CLASSES))
    y_cat = to_categorical(y, num_classes=NUM_CLASSES)
    return head,body,y_cat

def create_labels(data):

    from keras.utils.np_utils import to_categorical
    category_dict = {'unrelated': 0 , 'agree':1, 'disagree':2, 'discuss':3}
    y = list()
    NUM_CLASSES = 4
    for stance in data.stances:
        y.append(category_dict[stance['Stance']])

    y_cat = np.zeros((len(y),NUM_CLASSES))
    y_cat = to_categorical(y, num_classes=NUM_CLASSES)
    return y_cat



In [ ]:
(train_text,train_body,y_train) = create_dataset(train_dataset,None,True,t,max_head,max_body)
(test_text,test_body,y_test) = create_dataset(test_dataset,None,True,t,max_head,max_body)

# Conditional encoding model

In [ ]:
head_input = Input(shape=(max_head,), dtype='int32', name='head_input')
body_input = Input(shape=(max_body,), dtype='int32', name='body_input')
shared_embed = Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix],trainable=False)
head_embed = shared_embed(head_input)
body_embed = shared_embed(body_input)

body_lstm,state_h,state_c,x,y = Bidirectional(LSTM(100,dropout=0.2, recurrent_dropout=0.2, name='body_lstm',return_state=True))(body_embed)
encoded_states = [state_h,state_c,x,y]
head_lstm = Bidirectional(LSTM(100,dropout=0.2, recurrent_dropout=0.2, name='head_lstm'))(head_embed,initial_state=encoded_states)
dense = Dense(100,activation='relu')(head_lstm)
dense = Dropout(0.3)(dense)
dense = Dense(100,activation='relu')(dense)
dense = Dropout(0.3)(dense)
dense = Dense(100,activation='relu')(dense)
dense = Dropout(0.3)(dense)
dense = Dense(4,activation='softmax')(dense)
model = Model(inputs=[head_input,body_input], outputs=[dense])
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])
plot_model(model, to_file="drive/My Drive/Stance_detection/feed_forward_lstm.png")

# Conditional encoding model with summarization

In [ ]:
head_input = Input(shape=(max_head,), dtype='int32', name='head_input')
body_input = Input(shape=(max_body,), dtype='int32', name='body_input')
shared_embed = Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix],trainable=False)
head_embed = shared_embed(head_input)
body_embed = shared_embed(body_input)

body_lstm,state_h,state_c,x,y = Bidirectional(LSTM(100,dropout=0.2, recurrent_dropout=0.2, name='body_lstm',return_state=True))(body_embed)
encoded_states = [state_h,state_c,x,y]
head_lstm = Bidirectional(LSTM(100,dropout=0.2, recurrent_dropout=0.2, name='head_lstm'))(head_embed,initial_state=encoded_states)
dot_layer = dot([head_lstm,body_lstm],axes = 1, normalize=True)
conc = concatenate([head_lstm,body_lstm,dot_layer])
dense = Dense(100,activation='relu')(conc)
dense = Dropout(0.3)(dense)
dense = Dense(100,activation='relu')(dense)
dense = Dropout(0.3)(dense)
dense = Dense(100,activation='relu')(dense)
dense = Dropout(0.3)(dense)
dense = Dense(4,activation='softmax')(dense)
model = Model(inputs=[head_input,body_input], outputs=[dense])
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

# Model fit and save

In [ ]:
history = model.fit([train_text,train_body],[y_train],epochs=20,validation_split=0.2 ,batch_size=128,verbose = True)
model.save('drive/My Drive/Stance_detection/model/LSTM_feedforward.h5')


# Model Evaluate

In [ ]:
model.evaluate([test_text,test_body],[y_test])